## Import Required Packages

In [ ]:
import numpy as np
from girth.synthetic import create_synthetic_irt_dichotomous, create_synthetic_irt_polytomous
from girth_mcmc import GirthMCMC
from girth_mcmc.utils import tag_missing_data_mcmc, validate_mcmc_options

## Show the Processing Options

In [ ]:
print(validate_mcmc_options().keys())

## Run the 2PL model

In [ ]:
# Simulation Parameters
n_items = 25
n_people = 500

# Create Synthetic Data

discrimination = 0.89 * np.sqrt(-2 * np.log(np.random.rand(n_items)))
difficulty = np.random.randn(n_items)
theta = np.random.randn(n_people)

syn_data = create_synthetic_irt_dichotomous(difficulty, discrimination, 
                                            theta)

girth_model = GirthMCMC(model='2PL', 
                        options={'n_processors': 2,
                                 'n_tune': 5000,
                                 'n_samples': 5000})
results = girth_model(syn_data)


## Compare the results

In [ ]:
list(zip(discrimination.round(3), results['Discrimination'].round(3)))

## Run a Graded Response Model via Variational Inference

In [ ]:
# Simulation Parameters
n_items = 25
n_people = 500
n_levels = 3

# Create Synthetic Data
difficulty = np.sort(np.random.randn(n_items, n_levels), axis=1)
discrimination = np.random.rand(n_items) + 0.5
theta = np.random.randn(n_people)

syn_data = create_synthetic_irt_polytomous(difficulty, discrimination, theta, model='grm')

# Solve for parameters
girth_model = GirthMCMC(model='GRM', model_args=(n_levels+1,),
                        options={'variational_inference': True,
                                 'variational_samples': 10000,
                                 'n_samples': 10000,
                                 'n_processors': 2})
results = girth_model(syn_data)

# Print all the keys in the results
print(results.keys())

## Run with missing data

In [ ]:
# Create Some missing data
percent_missing = 10
missing_mask = np.random.rand(*syn_data.shape) < (percent_missing / 100)

missing_data = syn_data.copy()
missing_data[missing_mask] = 29

## Tag the missing data to process
missing_data = tag_missing_data_mcmc(missing_data, [1, 2, 3, 4])

# Solve for parameters
results_missing = girth_model(missing_data)

# Print all the keys in the results
print(results.keys())

## Show the difference

In [ ]:
list(zip(results_missing['Discrimination'].round(2), results['Discrimination'].round(2)))